In [ ]:
# import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D


# Get labels form csv file
train_all_csv = "drive/MyDrive/AI/train_all.csv"
df_train_all = pd.read_csv(train_all_csv)

test_csv = "drive/MyDrive/AI/test.csv"
df_test = pd.read_csv(test_csv)

# Numpy array will contain images
train_X = []
test_X = []

# DataFrame will contain labels
df_train_Y = []
df_test_Y = []

# Array that will be coverted to numpy array or dataFrame
train_X_array = np.load("drive/MyDrive/AI/np_train_X_all.npy")
train_Y_array = pd.read_csv("drive/MyDrive/AI/df_train_Y_all.csv")
test_X_array = []
test_Y_array = []

# List of predicted outputs
predictions_eth = []
predictions_gen = []
predictions_age = []

# Fixed values
IMG_SIZE = 200

In [ ]:
# Convert array to numpy array and DataFrame
train_X = np.array(train_X_array)
df_train_Y = pd.DataFrame(train_Y_array, columns=["ethnicity","gender","age"])
train_X.shape

In [ ]:
# Create model architecture
def base_model():

  visible = Input(shape=(IMG_SIZE,IMG_SIZE,3))
  conv21 = Conv2D(200, kernel_size=4, activation='relu')(visible)
  pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
  flat2 = Flatten()(pool21)

  hidden1 = Dense(128, activation='relu')(flat2)
  hidden2 = Dense(64, activation='relu')(hidden1)
  hidden3 = Dense(32, activation='relu')(hidden2)

  # Create output for ethnicity and age
  output_1 = Dense(3, name="ethnicity", activation="softmax")(hidden3)
  output_2 = Dense(2, name="gender", activation="softmax")(hidden3)

  # Create output for age
  hidden4 = Dense(64, activation='relu')(hidden3)
  hidden5 = Dense(128, activation='relu')(hidden4)
  output_3 = Dense(150, name="age", activation="softmax")(hidden5)

  model = Model(inputs=visible, outputs=[output_1,output_2,output_3])
  return model

model = base_model()
model.summary()

In [ ]:
# compile model
losses = {
    "ethnicity" : 'categorical_crossentropy',
    "gender" : 'categorical_crossentropy',
    "age" : 'categorical_crossentropy',
}

# lossWeights = {"ethnicity": 1.0, "gender": 1.0, "age": 1.0}

model.compile(loss=losses,
              # loss_weights = lossWeights,
              optimizer='adam', metrics=['accuracy'])

In [ ]:
eth = np.array(df_train_Y.ethnicity).astype("float32")
gen = np.array(df_train_Y.gender).astype("float32")
age = np.array(df_train_Y.age).astype("float32")

eth = tf.keras.utils.to_categorical(eth, 3)
gen = tf.keras.utils.to_categorical(gen, 2)
age = tf.keras.utils.to_categorical(age, 150)

In [ ]:
# Train model
model.fit(x=train_X, y={"ethnicity" : eth, "gender" : gen, "age" : age}, epochs=25)
